In [ ]:
!pip install kaggle

In [ ]:
import os
from google.colab import userdata

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# Create the config folder and file so the kaggle CLI works
!mkdir -p ~/.kaggle
!echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

print("Successfully configured Kaggle!")

In [ ]:
# PLANES VS DRONES VS BIRDS DATASET
# Dependencies:
import kagglehub
import os
import cv2 as OpenCV
import numpy

# Download the dataset
dataset_path = kagglehub.dataset_download("maryamlsgumel/drone-detection-dataset")

base_path = os.path.join(dataset_path, "BirdVsDroneVsAirplane")

# Navigate to the folder (update the path if the folder name is different)
data_dir = './drone-detection-dataset'

# important consts
data = []
labels = []
# hight and width of img after transformation
img_width = 64
img_height = 64

# Mapping folders to labels
# !!! category names need to be the same as folder names in the dataset because of how we travverse and translate data from directories!!!
categories = {'Birds': 0, 'Drones': 1, 'Aeroplanes': 2}

for folder_name, label in categories.items():
    # Use the path variable from kagglehub
    category_path = os.path.join(base_path, folder_name)

    if not os.path.isdir(category_path):
        print(f"Directory not found: {category_path}")
        continue

    print(f"Processing {folder_name}...")

    for img_name in os.listdir(category_path):
        # Skip system files and only keep images (skip thumb.db etc.)
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img_full_path = os.path.join(category_path, img_name)

            # Use OpenCV
            img = OpenCV.imread(img_full_path)

            if img is not None:
                # Resize to a standard size
                img = OpenCV.resize(img, (img_width, img_height))

                # Normalize pixels to range [0, 1]
                pixels = img.astype('float32') / 255.0

                data.append(pixels)
                labels.append(label)
        except Exception as e:
            print(f"Error loading {img_name}: {e}")

data_array = numpy.array(data)
labels_array = numpy.array(labels)

print(f"---")
print(f"Finished! Total images: {len(data_array)}")
print(f"data_array shape: {data_array.shape}") # (N, [width], [height], 3)
print(f"labels_array shape: {labels_array.shape}") # (N,)

In [ ]:
# display random img to check if eveything worked
import matplotlib.pyplot as plt
import random

# Pick a random index
idx = random.randint(0, len(data_array)-1)

# Display the image and its label
plt.imshow(data_array[idx])
plt.title(f"Label: {labels_array[idx]} (1:Bird, 2:Drone, 3:Plane)")
plt.axis('off')
plt.show()

In [ ]:
# we "flatten" the data to prepare it for training
# from a [width] [height] matrix to a line of numbers
# Calculate the total number of features per image
# [width] * [height] * 3 = all numbers/parameters in our array
# 3 represents primary colors Blue, Red, Green
n_samples = data_array.shape[0]
n_features = data_array.shape[1] * data_array.shape[2] * data_array.shape[3]

# Reshape to (Number of Images, Total Pixels)
features = data_array.reshape(n_samples, n_features)

print(f"Original Data Shape: {data_array.shape}")
print(f"Flattened 'features' Shape:  {features.shape}")

from sklearn.model_selection import train_test_split

# Show class distribution
unique_labels, counts = numpy.unique(labels, return_counts=True)
class_names = {0: "Birds", 1: "Drones", 2: "Aeroplanes"}
print(f"\nClass distribution:")
for label, count in zip(unique_labels, counts):
    print(f"  {class_names[label]}: {count} samples ({count/len(labels)*100:.1f}%)")

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"\nTrain/Test Split:")
print(f"  Training samples: {len(X_train)} ({len(X_train)/len(features)*100:.1f}%)")
print(f"  Testing samples: {len(X_test)} ({len(X_test)/len(features)*100:.1f}%)")
print(f"{'='*60}\n")

# importing time for evaluation
import time

In [18]:
# Evaluation Function:
# Most likely to change when the algorithm does get trained
# Pass the rows to the ml algorithms and check if the predicted result is correct, count that up and show % of success
def evaluate(model, x, y):

  # Use the trained model to predict the class for every row in x
  # x = the feature data (images flattened into numbers)
  predictions = model.predict(x)

  # Compare predictions with the real labels (y)
  # (predictions == y) creates an array of True/False values
  # numpy.sum counts how many True values (correct matches) there are
  correct = numpy.sum(predictions == y)

  # Total number of samples tested
  total = len(y)

  # Accuracy formula: correct predictions divided by total samples
  # Multiply by 100 to convert to percentage
  accuracy = (correct / total) * 100

  # Print evaluation results
  print("Correct:", correct)
  print("Total:", total)
  print("Accuracy: {:.2f}%".format(accuracy))

  # Return accuracy so it can also be stored or reused later
  return accuracy

In [25]:
# Algorythm 1 Euclidean Distance: looking at nearest neighbour and predicting as that class
class EuclideanDistance:
  def __init__(self):
      self.X_train = None
      self.y_train = None
      self.name = "Euclidean Distance"

  def train(self, X_train, y_train):
        print(f"\nTraining {self.name}...")
        self.X_train = X_train
        self.y_train = y_train
        print(f"Memorized {len(X_train)} training examples")

  def predict_single(self, x):
    # Calculate Euclidean distance to all training samples
      # Distance formula: sqrt(sum((x - train_sample)^2))
      distances = numpy.sqrt(numpy.sum((self.X_train - x)**2, axis=1))

      # Find the index of the smallest distance
      closest_index = numpy.argmin(distances)

      # Return the label of the closest training sample
      return self.y_train[closest_index]

  def predict(self, X_test):
    predictions = []
    for x in X_test:
        predictions.append(self.predict_single(x))
    return numpy.array(predictions)

# Algorythm 2 K Nearest Neighbours: looking at 3 nearest neighbours and predicting as most popular class among them
class KNearestNeighbors:
  def __init__(self, k=3):
        # k: Number of neighbors to vote (default 3)
        self.k = k
        self.X_train = None
        self.y_train = None
        self.name = f"K-Nearest Neighbors (k={k})"

  def train(self, X_train, y_train):
      print(f"\nTraining {self.name}...")
      self.X_train = numpy.array(X_train)
      self.y_train = numpy.array(y_train)
      print(f"Memorized {len(X_train)} training examples")

  def predict_single(self, x):
      # Calculate distances
      distances = numpy.sqrt(numpy.sum((self.X_train - x)**2, axis=1))

      # Get indices of K nearest neighbors
      # numpy.argpartition finds K smallest values efficiently
      k_indices = numpy.argpartition(distances, self.k)[:self.k].astype(int)

      # Get labels of K nearest neighbors
      k_labels = self.y_train[k_indices]

      # Count votes for each class
      # Find which class got the most votes
      unique_labels, counts = numpy.unique(k_labels, return_counts=True)
      winner = unique_labels[numpy.argmax(counts)]

      return winner

  def predict(self, X_test):
      predictions = []
      for x in X_test:
          predictions.append(self.predict_single(x))
      return numpy.array(predictions)


# Algorythm 3 Multi Layer Perceptron: A calssical neural network that trains x ammount of perceprtons over y epochs then predicts based on activated trained features
class MultiLayerPerceptron:
  def __init__(self, hidden_neurons=100, epochs=30, learning_rate=0.1, random_seed=42):
      self.hidden_neurons = hidden_neurons
      self.epochs = epochs
      self.learning_rate = learning_rate
      self.random_seed = random_seed
      self.name = f"Multi-Layer Perceptron ({hidden_neurons} neurons, {epochs} epochs)"

      # These will be set during training
      self.weights_input_hidden = None
      self.bias_hidden = None
      self.weights_hidden_output = None
      self.bias_output = None
      self.feature_mean = None
      self.feature_std = None
      self.n_classes = None
      self.input_size = None

  def sigmoid(self, x):
      # Sigmoid activation function: squashes values to range (0, 1)
      # Formula: 1 / (1 + e^(-x))
      return 1.0 / (1.0 + numpy.exp(-numpy.clip(x, -500, 500)))

  def sigmoid_derivative(self, activated_value):
      # Derivative of sigmoid - used in backpropagation
      # Formula: sigmoid(x) * (1 - sigmoid(x))

      return activated_value * (1.0 - activated_value)

  def normalize_features(self, X):
      # Normalize features to have mean=0, std=1
      if self.feature_mean is None:
          # First time - calculate mean and std from training data
          self.feature_mean = numpy.mean(X, axis=0)
          self.feature_std = numpy.std(X, axis=0)
          # Prevent division by zero
          self.feature_std[self.feature_std < 1e-9] = 1.0

      # Apply normalization
      return (X - self.feature_mean) / self.feature_std

  def initialize_weights(self):
      # Initialize weights randomly
      numpy.random.seed(self.random_seed)
      weight_range = 0.1

      # Weights from input to hidden layer
      self.weights_input_hidden = (numpy.random.rand(self.hidden_neurons, self.input_size) * 2 - 1) * weight_range
      self.bias_hidden = (numpy.random.rand(self.hidden_neurons) * 2 - 1) * weight_range

      # Weights from hidden to output layer
      self.weights_hidden_output = (numpy.random.rand(self.n_classes, self.hidden_neurons) * 2 - 1) * weight_range
      self.bias_output = (numpy.random.rand(self.n_classes) * 2 - 1) * weight_range

  def forward_pass(self, x):
      """
      Forward pass: push data through the network

      Steps:
      1. Input -> Hidden layer (with sigmoid activation)
      2. Hidden -> Output layer (with sigmoid activation)
      """
      # Input to hidden layer
      hidden_sum = numpy.dot(self.weights_input_hidden, x) + self.bias_hidden
      hidden_activation = self.sigmoid(hidden_sum)

      # Hidden to output layer
      output_sum = numpy.dot(self.weights_hidden_output, hidden_activation) + self.bias_output
      output_activation = self.sigmoid(output_sum)

      return hidden_activation, output_activation

  def train(self, X_train, y_train):
      """
      Train the neural network using backpropagation

      Backpropagation:
      1. Make a prediction (forward pass)
      2. Calculate how wrong we were (error)
      3. Adjust weights to reduce error (backward pass)
      4. Repeat many times
      """
      print(f"\nTraining {self.name}...")
      print("This may take a minute...")

      # Get dimensions
      n_samples, self.input_size = X_train.shape
      self.n_classes = len(numpy.unique(y_train))

      # Normalize features
      X_normalized = self.normalize_features(X_train)

      # Initialize weights
      self.initialize_weights()

      # Training loop
      for epoch in range(self.epochs):
          correct = 0

          # Go through each training example
          for i in range(n_samples):
              x = X_normalized[i]
              target_class = y_train[i]

              # FORWARD PASS: Make a prediction
              hidden, output = self.forward_pass(x)

              # Create target vector (one-hot encoding)
              # Example: if target_class=1 and n_classes=3, target=[0, 1, 0]
              target = numpy.zeros(self.n_classes)
              target[target_class] = 1.0

              # CALCULATE ERROR
              # Output layer error
              output_error = target - output
              output_delta = output_error * self.sigmoid_derivative(output)

              # Hidden layer error (backpropagate)
              hidden_error = numpy.dot(self.weights_hidden_output.T, output_delta)
              hidden_delta = hidden_error * self.sigmoid_derivative(hidden)

              # UPDATE WEIGHTS (gradient descent)
              # Hidden to output weights
              self.weights_hidden_output += self.learning_rate * numpy.outer(output_delta, hidden)
              self.bias_output += self.learning_rate * output_delta

              # Input to hidden weights
              self.weights_input_hidden += self.learning_rate * numpy.outer(hidden_delta, x)
              self.bias_hidden += self.learning_rate * hidden_delta

              # Track accuracy
              if numpy.argmax(output) == target_class:
                  correct += 1

          # Print progress every 10 epochs
          if (epoch + 1) % 10 == 0:
              accuracy = (correct / n_samples) * 100
              print(f"  Epoch {epoch + 1}/{self.epochs} - Training accuracy: {accuracy:.2f}%")

      print("Training complete!")

  def predict_single(self, x):
      _, output = self.forward_pass(x)
      return numpy.argmax(output)

  def predict(self, X_test):
      # Normalize test data using training statistics
      X_normalized = (X_test - self.feature_mean) / self.feature_std

      predictions = []
      for x in X_normalized:
          predictions.append(self.predict_single(x))

      return numpy.array(predictions)



In [26]:
results = {}

# Test all algorithms
euclidean = EuclideanDistance()
euclidean.train(X_train, y_train)
results['Euclidean'] = evaluate(euclidean, X_test, y_test)

knn3 = KNearestNeighbors(k=3)
knn3.train(X_train, y_train)
results['KNN-3'] = evaluate(knn3, X_test, y_test)

knn5 = KNearestNeighbors(k=5)
knn5.train(X_train, y_train)
results['KNN-5'] = evaluate(knn5, X_test, y_test)

mlp = MultiLayerPerceptron(hidden_neurons=100, epochs=30)
mlp.train(X_train, y_train)
results['MLP'] = evaluate(mlp, X_test, y_test)

# Show results
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
for rank, (name, acc) in enumerate(sorted_results, 1):
    print(f"{rank}. {name}: {acc:.2f}%")


Training Euclidean Distance...
Memorized 1884 training examples
Correct: 392
Total: 472
Accuracy: 83.05%

Training K-Nearest Neighbors (k=3)...
Memorized 1884 training examples
Correct: 332
Total: 472
Accuracy: 70.34%

Training K-Nearest Neighbors (k=5)...
Memorized 1884 training examples
Correct: 333
Total: 472
Accuracy: 70.55%

Training Multi-Layer Perceptron (100 neurons, 30 epochs)...
This may take a minute...
  Epoch 10/30 - Training accuracy: 79.94%
  Epoch 20/30 - Training accuracy: 83.17%
  Epoch 30/30 - Training accuracy: 85.83%
Training complete!
Correct: 360
Total: 472
Accuracy: 76.27%
1. Euclidean: 83.05%
2. MLP: 76.27%
3. KNN-5: 70.55%
4. KNN-3: 70.34%
